<a href="https://colab.research.google.com/github/ErnstHolger/Machine-Learning-For-Manufacturing/blob/main/Data_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q stumpy

     |████████████████████████████████| 136 kB 5.5 MB/s 


In [2]:
from IPython.display import Image,display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from time import sleep
from os.path import exists

if not exists('/content/Machine-Learning-For-Manufacturing/'):
  ! git clone https://github.com/ErnstHolger/Machine-Learning-For-Manufacturing
! chmod ogu+rwx /content/Machine-Learning-For-Manufacturing/data/*.*

Cloning into 'Machine-Learning-For-Manufacturing'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 3), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (26/26), done.
chmod: cannot access '/content/Machine-Learning-For-Manufacturing/data/*.*': No such file or directory


In [42]:
import pandas as pd
import stumpy

import plotly
import plotly.graph_objects as go
from plotly.graph_objs import Scatter, Layout
from plotly.subplots import make_subplots

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="Courier New",
    font_size=12,
    font_color="#a5b1cd",
    title_font_family="Courier New",
    title_font_color="black",
    title_font_size=12,
    uirevision=True,
    autosize=True
)

o=pd.read_csv('/content/Machine-Learning-For-Manufacturing/Data/o.csv')
pc=pd.read_csv('/content/Machine-Learning-For-Manufacturing/Data/pc.csv')
pi=pd.read_csv('/content/Machine-Learning-For-Manufacturing/Data/pi.csv')

# 13:00 is local time!
o['TimeStamp']=pd.to_datetime(o['TimeStamp'].values)
pc.head()
pc['TimeStamp']=pd.to_datetime(pc['LocalDateTime'].values)
pi['TimeStamp']=pd.to_datetime(pi['LocalDateTime'].values)



In [51]:
# need to resample
if o.index.name!='TimeStamp':
  o.set_index('TimeStamp',inplace=True)
  pc.set_index('TimeStamp',inplace=True)

o=o.resample('1S').interpolate(method='linear')
pc=pc.resample('1S').interpolate(method='linear')

# alignment using stumpy
distance_profile = stumpy.mass(o['Value'].values,pc['Value'].values,normalize=False)
index_min = min(range(len(distance_profile)), key=distance_profile.__getitem__)
n=len(o['Value'].values)

In [53]:
# raw, similarity, aligned
fig = make_subplots(rows=3, cols=1, shared_xaxes=False)
fig.update_layout(layout)

fig.add_trace(go.Scatter(x=pc.index,y=pc['Value'], name='pc'), row=1, col=1)
fig.add_trace(go.Scatter(x=o.index,y=o['Value'], name='o'),row=1, col=1)

fig.add_trace(go.Scatter(y=distance_profile, name='distance_profile'),row=2, col=1)

fig.add_trace(go.Scatter(y=o['Value'].values, name='o-aligned'),row=3, col=1)
fig.add_trace(go.Scatter(y=pc['Value'].values[index_min:index_min+n], name='pc-aligned'),row=3, col=1)
fig.show()

In [61]:
# some metrics
from sklearn.metrics import *

y_true=o['Value'].values
y_pred=pc['Value'].values[index_min:index_min+n]

print('max_error: ',max_error(y_true, y_pred))
print('explained_variance_score: ',explained_variance_score(y_true, y_pred))
print('mean_absolute_error: ',mean_absolute_error(y_true, y_pred))
print('mean_squared_error: ',mean_squared_error(y_true, y_pred))
print('median_absolute_error ',median_absolute_error(y_true, y_pred))

max_error:  0.10528173913043304
explained_variance_score:  0.9994160188890214
mean_absolute_error:  0.024457308527517315
mean_squared_error:  0.0010573410749903382
median_absolute_error  0.018780025466895722
